# TM10007: Machine learning
## Week 1, lecture 2: Generalization
#### Author: Hakim C. Achterberg

In this exercise, you will have a look at generalization and experimental setup.

For documentation about how to implement these methods in Python have a look at: https://scikit-learn.org/stable/model_selection.html


In [0]:
!pip install sklearn numpy matplotlib

In [0]:
# General packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
import seaborn

# Classifiers
from sklearn import model_selection
from sklearn import metrics


In [0]:
# Some functions we will use
from sklearn.decomposition import PCA

def colorplot(clf, ax, x, y, h=100):
    '''
    Overlay the decision areas as colors in an axes.
    
    Input:
        clf: trained classifier
        ax: axis to overlay color mesh on
        x: feature on x-axis
        y: feature on y-axis
        h(optional): steps in the mesh
    '''
    # Create a meshgrid the size of the axis
    xstep = (x.max() - x.min() ) / 20.0
    ystep = (y.max() - y.min() ) / 20.0
    x_min, x_max = x.min() - xstep, x.max() + xstep
    y_min, y_max = y.min() - ystep, y.max() + ystep
    h = max((x_max - x_min, y_max - y_min))/h
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    if hasattr(clf, "decision_function"):
        Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    else:
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    if len(Z.shape) > 1:
        Z = Z[:, 1]
    
    # Put the result into a color plot
    cm = plt.cm.RdBu_r
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)
    del xx, yy, x_min, x_max, y_min, y_max, Z, cm
    
def load_breast_cancer(n_features=2):
    '''
    Load the sklearn breast data set, but reduce the number of features with PCA.
    '''
    data = ds.load_breast_cancer()
    x = data['data']
    y = data['target']
    
    p = PCA(n_components=n_features)
    p = p.fit(x)
    x = p.transform(x)
    return x, y

def load_boston(n_features=1):
    '''
    Load the sklearn boston data set, but reduce the number of features with PCA.
    '''
    data = ds.load_boston()
    x = data['data']
    y = data['target']
    
    p = PCA(n_components=n_features)
    p = p.fit(x)
    x = p.transform(x)
    return x, y

def load_diabetes(n_features=1):
    '''
    Load the sklearn bdiabetes data set, but reduce the number of features with PCA.
    '''
    data = ds.load_diabetes()
    x = data['data']
    y = data['target']
    
    p = PCA(n_components=n_features)
    p = p.fit(x)
    x = p.transform(x)
    return x, y

Let us first create again three example datasets to play with and plot the feature distributions
in scatter plots.

In [0]:
# Create a noise dataset
X2, y2 = ds.make_moons(n_samples=1000, noise=0.4, random_state=0)

# Add extra external validation set
X3, y3 = ds.make_moons(n_samples=5000, noise=0.4, random_state=42)


fig = plt.figure(figsize=(24,8))
ax = fig.add_subplot(131)
ax.set_title("Noisy moon, entire dataset", fontsize='small')
ax.scatter(X2[:, 0], X2[:, 1], marker='o', c=y2,
           s=25, edgecolor='k', cmap=plt.cm.Paired)

# Split the dataset in train and test part
X2_train, X2_test, y2_train, y2_test = model_selection.train_test_split(X2, y2, test_size=0.5, stratify=y2)

ax = fig.add_subplot(132)
ax.set_title("Training data", fontsize='small')
ax.scatter(X2_train[:, 0], X2_train[:, 1], marker='o', c=y2_train,
           s=25, edgecolor='k', cmap=plt.cm.Paired)

ax = fig.add_subplot(133)
ax.set_title("Test data", fontsize='small')
ax.scatter(X2_test[:, 0], X2_test[:, 1], marker='o', c=y2_test,
           s=25, edgecolor='k', cmap=plt.cm.Paired)


## Attempt at classification using a kNN

As can be seen the generated dataset has a non-linear decision boundary and has some class overlap. This makes the classification problem harder because simple models will fail to find a good separation. A simple linear boundary could not separate the classes very well.

We will attempt to solve this problem with a kNN classifier. Let's train a few k-Nearest Neighbor classifiers and see the performance.

In [0]:
from sklearn import neighbors
k_list = [1, 3, 7]
fig = plt.figure(figsize=(24,8*len(k_list)))
num = 0
    
for k in k_list:
    clf_knn = neighbors.KNeighborsClassifier(n_neighbors=k)
    clf_knn.fit(X2_train, y2_train)

    # Test the classifier on the training data and plot
    score_train = clf_knn.score(X2_train, y2_train)

    num += 1
    ax = fig.add_subplot(len(k_list), 2, num)
    ax.set_title(f"Training performance: accuracy {score_train}")
    colorplot(clf_knn, ax, X2[:, 0], X2[:, 1], h=1000)
    ax.scatter(X2_train[:, 0], X2_train[:, 1], marker='o', c=y2_train,
               s=25, edgecolor='k', cmap=plt.cm.Paired)

    # Test the classifier on the test data and plot
    score_test = clf_knn.score(X2_test, y2_test)

    num += 1
    ax = fig.add_subplot(len(k_list), 2, num)
    ax.set_title(f"Test performance: accuracy {score_test}")
    colorplot(clf_knn, ax, X2[:, 0], X2[:, 1], h=1000)
    ax.scatter(X2_test[:, 0], X2_test[:, 1], marker='o', c=y2_test,
               s=25, edgecolor='k', cmap=plt.cm.Paired)
    


You can clearly see that the 1-NN looks perfect on the training data but seen to not perform as well on the test data. We can conclude that it doesn't generalize very well. The 3-NN and 7-NN seem to perform worse on the training data, but better on the test data. This indicates that these classifiers are less prone to over-fitting and generalize better.

# Gauging generalizability

We can try to see how much regularization (higher k) helps in for this dataset by plotting a curve. This might give un an idea for different parameters, how well the model generalizes.

In [0]:
from sklearn import neighbors
    
train_scores = []
test_scores = []
k_list = list(range(1, 25, 2))

for k in k_list:
    clf_knn = neighbors.KNeighborsClassifier(n_neighbors=k)
    clf_knn.fit(X2_train, y2_train)

    # Test the classifier on the training data and plot
    score_train = clf_knn.score(X2_train, y2_train)
    score_test = clf_knn.score(X2_test, y2_test)
    
    train_scores.append(score_train)
    test_scores.append(score_test)
    
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.grid()
ax.plot(k_list, train_scores, 'o-', color="r",
        label="Training score")
ax.plot(k_list, test_scores, 'o-', color="g",
        label="Test score")

You can see the lines converging around k=13, after that they fluctuate but stay fairly similar. It's hard to draw conclusions from a single point/line, because we do not have any confidence or statistical tests. But the line does give an indication that around k=15 seems decent value. To get a more clear idea, instead of a single split, we could perform a repeat random split and see the distribution of scores. This is a bit more work:

In [0]:
k_list = list(range(1, 26, 2))
all_train = []
all_test = []

# Repeat the experiment 20 times, use 20 random splits in which class balance is retained
sss = model_selection.StratifiedShuffleSplit(n_splits=20, test_size=0.5, random_state=0)

for train_index, test_index in sss.split(X2, y2):
    train_scores = []
    test_scores = []
    
    split_X_train = X2[train_index]
    split_y_train = y2[train_index]
    split_X_test = X2[test_index]
    split_y_test = y2[test_index]

    for k in k_list:
        clf_knn = neighbors.KNeighborsClassifier(n_neighbors=k)
        clf_knn.fit(split_X_train, split_y_train)

        # Test the classifier on the training data and plot
        score_train = clf_knn.score(split_X_train, split_y_train)
        score_test = clf_knn.score(split_X_test, split_y_test)

        train_scores.append(score_train)
        test_scores.append(score_test)
        
    all_train.append(train_scores)
    all_test.append(test_scores)
    

# Create numpy array of scores and calculate the mean and std
all_train = np.array(all_train)
all_test = np.array(all_test)

train_scores_mean = all_train.mean(axis=0)
train_scores_std = all_train.std(axis=0)

test_scores_mean = all_test.mean(axis=0)
test_scores_std = all_test.std(axis=0)

# Plot the mean scores and the std as shading
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.grid()
ax.fill_between(k_list, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
ax.fill_between(k_list, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1,
                     color="g")
ax.plot(k_list, train_scores_mean, 'o-', color="r",
        label="Training score")
ax.plot(k_list, test_scores_mean, 'o-', color="g",
        label="Test score")

### Use AUC instead of Accuracy

Adapt the code to display the AUC instead of the accuracy. You can use the following function to calculate the AUC: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score

To obtain the scores argument for the AUC, you can use the predict_proba method of the kNN classifier.

In [0]:
k_list = list(range(1, 26, 2))
all_train = []
all_test = []

# Repeat the experiment 20 times, use 20 random splits in which class balance is retained
sss = model_selection.StratifiedShuffleSplit(n_splits=20, test_size=0.5, random_state=0)

for train_index, test_index in sss.split(X2, y2):
    train_scores = []
    test_scores = []
    
    split_X_train = X2[train_index]
    split_y_train = y2[train_index]
    split_X_test = X2[test_index]
    split_y_test = y2[test_index]

    for k in k_list:
        clf_knn = neighbors.KNeighborsClassifier(n_neighbors=k)
        clf_knn.fit(split_X_train, split_y_train)

        # Test the classifier on the training data and plot
        train_proba = clf_knn.predict_proba(split_X_train)[:, 1]
        test_proba = clf_knn.predict_proba(split_X_test)[:, 1]
        
        score_train = metrics.roc_auc_score(split_y_train, train_proba)
        score_test = metrics.roc_auc_score(split_y_test, test_proba)
        

        train_scores.append(score_train)
        test_scores.append(score_test)
        
    all_train.append(train_scores)
    all_test.append(test_scores)
    

# Create numpy array of scores and calculate the mean and std
all_train = np.array(all_train)
all_test = np.array(all_test)

train_scores_mean = all_train.mean(axis=0)
train_scores_std = all_train.std(axis=0)

test_scores_mean = all_test.mean(axis=0)
test_scores_std = all_test.std(axis=0)

# Plot the mean scores and the std as shading
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.grid()
ax.fill_between(k_list, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
ax.fill_between(k_list, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1,
                     color="g")
ax.plot(k_list, train_scores_mean, 'o-', color="r",
        label="Training score")
ax.plot(k_list, test_scores_mean, 'o-', color="g",
        label="Test score")

# Setting up an experiment for kNN classification

Now imagine we want to do a proper experiment. We want to:
* automatically find the optimal k hyperparameter
* estimate performance correctly
* have an indication of generalizability

For this we would need to split the datain more than two parts:
* train set for fitting models
* validation set for testing hyperparamters (the different k values)
* test set for the final evaluation

To limit the sample sets getting small, we would like to use cross validation.

## Fitting a k-NN  with optimal k
There are methods in scikit-learn to easily find the optimal hyper parameters using gridsearch and cross-validation. You can specify the classifier, parameters to search, cross-validation method and scoring measure yourself in an easy way:

In [0]:
# Create a grid search to find the optimal k using a gridsearch and 10-fold cross validation

# Specify the classifier
knn = neighbors.KNeighborsClassifier()

# Specify the search range, this could be multiple parameters for more complex classifiers
parameters = {
    "n_neighbors": list(range(1, 26, 2))
}

# Specify the cross validation method to use, we use 10-fold stratified cross-validation
cv_10fold = model_selection.StratifiedKFold(n_splits=10)

# Create the grid search method, use area under ROC curve as scoring metric
# Too learn more about metrics see: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
grid_search = model_selection.GridSearchCV(knn, parameters, cv=cv_10fold, scoring='roc_auc')

# Do the entire search
grid_search.fit(X2_train, y2_train)

# Show the complete results of the cross validation
pd.DataFrame(grid_search.cv_results_)

### Change the CV and metric options

You can play around with the above example. You could change the metrics use from AUC to F-score or another metric. How does this influence the resutsl? What about changing from a stratified to an unstratified cross-validation?

## Perform the grid search in a cross-validation setting

We will now wrap the previous grid-search example in another cross validation loop. In this loop we will split the data, fit the classifier, score the test data with the optimal classifier.

In [0]:
# Create a 20 fold stratified CV iterator
cv_20fold = model_selection.StratifiedKFold(n_splits=10)
results = []
best_n_neighbors = []

# Loop over the folds
for validation_index, test_index in cv_20fold.split(X2, y2):
    # Split the data properly
    X_validation = X2[validation_index]
    y_validation = y2[validation_index]
    
    X_test = X2[test_index]
    y_test = y2[test_index]
    
    # Create a grid search to find the optimal k using a gridsearch and 10-fold cross validation
    # Same as above
    parameters = {"n_neighbors": list(range(1, 26, 2))}
    knn = neighbors.KNeighborsClassifier()
    cv_10fold = model_selection.StratifiedKFold(n_splits=10)
    grid_search = model_selection.GridSearchCV(knn, parameters, cv=cv_10fold, scoring='roc_auc')
    grid_search.fit(X_validation, y_validation)
    
    # Get resulting classifier
    clf = grid_search.best_estimator_
    print(f'Best classifier: k={clf.n_neighbors}')
    best_n_neighbors.append(clf.n_neighbors)
    
    # Test the classifier on the test data
    probabilities = clf.predict_proba(X_test)
    scores = probabilities[:, 1]
    
    # Get the auc
    auc = metrics.roc_auc_score(y_test, scores)
    results.append({
        'auc': auc,
        'k': clf.n_neighbors,
        'set': 'test'
    })
    
    # Test the classifier on the validation data
    probabilities_validation = clf.predict_proba(X_validation)
    scores_validation = probabilities_validation[:, 1]
    
    # Get the auc
    auc_validation = metrics.roc_auc_score(y_validation, scores_validation)
    results.append({
        'auc': auc_validation,
        'k': clf.n_neighbors,
        'set': 'validation'
    })
    
# Create results dataframe and plot it
results = pd.DataFrame(results)
seaborn.boxplot(y='auc', x='set', data=results)

optimal_n = int(np.median(best_n_neighbors))
print(f"The optimal N={optimal_n}")

### Extra steps

If you would like to add another step to the method (e.g. feature scaling), where would you place that in the above example? How would you implement this? This will be the topic of next weeks exercises.

## Replication experiment

The result of the classification looks fairly good, we have a high AUC and test van validation appear to be not too far apart. If we want to test the generalization more, we could use an independent replication set. We could quickly see if that works by fitting the method on the current data and applying to the new set.

In [0]:
# Use the optimal parameters without any tuning to validate the optimal classifier
clf = neighbors.KNeighborsClassifier(n_neighbors=optimal_n)

# Fit on the entire dataset
clf.fit(X2, y2)

# Test the classifier on the indepedent replication data
probabilities = clf.predict_proba(X3)
scores = probabilities[:, 1]

# Get the auc
auc = metrics.roc_auc_score(y3, scores)
print(f'THe AUC on the replication set is {auc} using a {clf.n_neighbors}-NN classifier')